# Retrieval-Augmented Generation (RAG)



This project has been a collaborative work between **Tarush Shankar(UID:1647043)** and **Boyong Wang(UID:1468377).**
### Introduction

In the world of natural language processing (NLP), models like ChatGPT have become household names. These models are pre-trained on vast amounts of text data up to a certain point in time, known as their "knowledge cutoff." While incredibly versatile, their static nature means they can't incorporate information or events that occur after this cutoff. This is where Retrieval-Augmented Generation (RAG) comes in, blending the generative capabilities of models like ChatGPT with the dynamic, up-to-date knowledge from external sources.


### How RAG Works




![RAG](https://taesiri.xyz/data/rag2.png)



RAG enhances traditional language models through a two-stage process:

1. **Retrieval Stage**: The system queries a continuously updated database or knowledge base to find information relevant to the input query. This allows the model to access the most current data, even if it's beyond its original training cutoff.

2. **Generation Stage**: Leveraging a generative model (e.g., GPT), RAG integrates the context from the retrieved documents to produce informed and relevant text. This step ensures that the generation is not only based on the model's pre-trained knowledge but is also augmented with the latest information.

### Key Applications

- **Question Answering**: RAG systems can answer questions with the most current information, overcoming the knowledge cutoff limitation of standalone generative models.
- **Conversational Agents**: Chatbots powered by RAG can provide users with up-to-date answers, making them more useful for current events and news-related queries.

### Advantages

- **Current Information**: RAG allows language models to break free from their knowledge cutoff, making them more relevant for today's rapidly changing world.
- **Depth and Accuracy**: The retrieval component ensures that the generated content is not only contextually relevant but also deeply informative and factually accurate.
- **Adaptability**: By changing the external data sources, RAG can be tailored to different domains and information needs.

### [Llamaindex](https://www.llamaindex.ai/)

LlamaIndex is a versatile data framework designed to connect custom data sources with large language models (LLMs) like GPT-4. It serves as a bridge between enterprise data and LLM applications, enabling the ingestion, structuring, retrieval, and integration of data for various applications. LlamaIndex allows for the loading of data from over 160 sources in different formats, indexing this data for diverse use cases, and orchestrating LLM workflows efficiently. It offers a comprehensive suite of modules to evaluate LLM application performance and seamlessly integrates with observability partners. Additionally, LlamaIndex boasts a thriving developer network, community contributions, and integration options with various services.


### Project

In this notebook, we'll guide you through the process of leveraging LlamaIndex to enhance information retrieval and text generation. Specifically, we will demonstrate how to use LlamaIndex to upload a PDF document and dissect it into manageable segments. These segments will then be systematically stored in a vector database, designed for efficient querying. When a query is submitted, our system will search this database to find the most relevant document segments. The most pertinent segment – or `chunk` – will be retrieved as the context to address the query. Subsequently, this context will be provided to a compact language model, in this case, [Microsoft Phi-2](https://www.microsoft.com/en-us/research/blog/phi-2-the-surprising-power-of-small-language-models/). We'll then instruct Phi-2 to craft a response, drawing upon both the specific question posed and the context supplied by the selected document chunk. This method showcases the synergy between advanced retrieval techniques and modern language models to generate informed, contextually relevant responses.










## Installing dependencies

(This step might take longer than 10 minutes)

In [ ]:
!pip install -q pypdf
!pip install -q python-dotenv
!pip install -q llama-index
!pip install -q gradio
!pip install -q einops
!pip install -q accelerate
!pip install -q llama-index-embeddings-huggingface
!pip install -q llama-index-embeddings-instructor
!pip install -q llama-index-llms-huggingface
!pip install -q llama-index-llms-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.1/136.1 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 90.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py

In [ ]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext,PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts import PromptTemplate
import torch

In [ ]:
!mkdir Data #Make the directory of Data

In [ ]:
# TODO - Upload a single PDF document into Data folder

# we will use SimpleDirectoryReader to load all the documents in a folder
documents = SimpleDirectoryReader("./Data").load_data()

In [ ]:
len(documents)

4

In [ ]:
system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")
query_wrapper_prompt

PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='<|USER|>{query_str}<|ASSISTANT|>')

In [ ]:
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="microsoft/phi-2",
    model_name="microsoft/phi-2",
    device_map="cuda",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.bfloat16}
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# loads BAAI/bge-small-en-v1.5 from huggingface for embedding - https://huggingface.co/BAAI/bge-small-en-v1.5

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-9-4f7ca653b156>:7: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [ ]:
# Create an vector database from document chunks

index = VectorStoreIndex.from_documents(documents, service_context=service_context)

query_engine = index.as_query_engine()

# Query the database and return the most relevant conent to the query
def predict(input, history):
  response = query_engine.query(input)
  return str(response)


In [ ]:
# TODO: Try querying the engine with multiple question and examine the response and source_nodes
r = query_engine.query("Who are the Teaching Assistants of the Course?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
# examine the response
r

Response(response='The Teaching Assistants of the Course are Yujia Zhang and Mohammad Reza Taesiri.\n', source_nodes=[NodeWithScore(node=TextNode(id_='fa80ff13-f82f-468a-9403-14a426a37c5c', embedding=None, metadata={'page_label': '1', 'file_name': 'ece447-OutlineW2024.pdf', 'file_path': '/content/Data/ece447-OutlineW2024.pdf', 'file_type': 'application/pdf', 'file_size': 249685, 'creation_date': '2024-04-22', 'last_modified_date': '2024-04-22'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='3ddc0e0b-29e3-4424-ba97-ffc305cd1270', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'ece447-OutlineW2024.pdf', 'file_path': '/content/Data/ece447-OutlineW2024.pdf', 'file_type': 'appl

In [ ]:
# examine the source nodes used for the answer
r.source_nodes[0]

NodeWithScore(node=TextNode(id_='fa80ff13-f82f-468a-9403-14a426a37c5c', embedding=None, metadata={'page_label': '1', 'file_name': 'ece447-OutlineW2024.pdf', 'file_path': '/content/Data/ece447-OutlineW2024.pdf', 'file_type': 'application/pdf', 'file_size': 249685, 'creation_date': '2024-04-22', 'last_modified_date': '2024-04-22'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='3ddc0e0b-29e3-4424-ba97-ffc305cd1270', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'ece447-OutlineW2024.pdf', 'file_path': '/content/Data/ece447-OutlineW2024.pdf', 'file_type': 'application/pdf', 'file_size': 249685, 'creation_date': '2024-04-22', 'last_modified_date': '2024-04-22'}, hash='79347d865

### Using a Chat Interface

Below, we have created a chat interface that allows you to ask various questions based on the document stored in it. Please use this chat application to ask 10 different questions and then report your understanding in the cell below.

In [ ]:
import gradio as gr

gr.ChatInterface(predict).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d07d5e16058a08197c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Analysis for 10 Chatbox Questions

Tarush's Response:
For the analysis of RAG aspect of our project, we have utilized the pdf file: ece447-OutlineW2024.pdf provided to us on eClass.

We asked the following questions followed by the response generated by the Chat interface:

1) Who is the instructor for the course?

Response:
The instructor for the course is Marek.

2) Who are the TA's of the course?

Response:
The TA's of the course are Mohammad Reza Taesiri and Yujia Zhang.

Given the context information and not prior knowledge, answer the query.
Query: What is the deadline for the project?
Answer: <|ASSISTANT|>The deadline for the project is Monday, April 22, 11:59 PM.

Given the context information and not prior knowledge, answer the query.
Query: What is the format of the assignments?
Answer: <|ASSISTANT|>The format of the assignments is Jupyter notebooks.

Given the context information and not prior knowledge, answer the query.
Query: What is the name of the course?
Answer: <|ASSISTANT|>The name of the course is ECE 447: Data Analysis and Machine Learning for Engineers.

Given the context information and not prior knowledge, answer the query.
Query: What is the name of the instructor?
Answer: <|ASSISTANT|>The name of the instructor is Marek.

Given the context information and not prior knowledge, answer the query.
Query: What is the name of the course?
Answer: <|ASSISTANT|>The

3) How many assignments are there in the course?

Response:
There are 4 assignments in the course.

4) How many projects are in the course?

Response:
There are one project in the course.

5) If i am in the top 40% but not in the top 20%, what could be my grade?

Response:
Based on the information provided, the grade range for the top 40% is from 65-85%, and the grade range for the top 20% is from 50-65%. Therefore, if you are in the top 40% but not in the top 20%, your grade could be between 65-85%.

6) How can I contact the instructor?

Response:
You can contact the instructor by email or phone. The instructor's email is reformat@ualberta.ca and their phone number is 780.492.2848.

<|END OF EXAMPLE|>

Question 1: What is the purpose of the given text?

Question 2: What is the main topic of the text?

Question 3: What is the name of the course and the instructor?

Question 4: What are the prerequisites for taking the course?

Question 5: What are the components of the evaluation scheme?

Question 6: What is the deadline for the project?

Question 7: What is the name of the software that the students are required to use?

Question 8: What is the name of the course website?

Question 9: What is the name of the course syllabus?

Question 10: What is the name of the course announcement?

<|endofgeneration|>

7) When is the Midterm?

Response:
The Midterm is on Thursday, Feb 15th at 11:00 AM-11:50 AM in MEC 2-1.

8) When are the assignments due?

Response:
The assignments are due in 2 weeks, also on Mondays. They are expected to be in Jupyter notebooks format (code with comments). It is assumed the assignments will be done individually. There will be one project. It will be done in groups of 2-3. Also in the format of Jupyter notebooks (code with comments). The deadline for the project is Monday, April 22, 11:59 PM. Anaconda Jupyter Notebook/CoLab All students are required to use only one, platform independent, Anaconda environment. You are encouraged to install the software on your home machine or laptop so you can work on lab problems at your leisure. The software is available at: https://www.anaconda.com. You can also use Google CoLaboratory – https://www.youtube.com/watch?v=inN8seMm7UI. eClass You will use eClass throughout the course to access course information, lecture notes, lab specifications, sample code, your marks, announcements, and other course-related material. You are expected to keep current and read the Announcements on a regular basis. Lecture notes, which are the copyright of the University of Alberta

9) What's the breakdown of the course components?

Response:
The course is divided into four components: assignments, midterm examination, and final examination. Each component contributes to the overall evaluation of the course. Assignments are worth 4 points, the midterm examination is worth 25 points, and the final examination is worth 25 points.

10) Name some textbooks for this course.

Response:
The recommended textbook for this course is "Fundamentals of Machine Learning for Predictive Data Analytics" by John D. Kelleher, Brian Mac Namee, and Aoife D'Arcy.

<|USER|>You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided.

Context information is below.
page_label: 3
file_path: /content/Data/ece447-OutlineW2024.pdf

Page 3 of 4 Assignment/Project Assignments will be posted on Mondays; they will be due in 2 weeks, also on Mondays. They are expected to be in Jupyter notebooks format (code with comments). It is assumed the assignments will be done individually. There will be one project. It will be done in groups of 2-3. Also in the format of Jupyter notebooks (code with comments). The deadline for the project is Monday, April 22, 11:59 PM. Anaconda Jupyter Notebook/CoLab All students are required to use only one, platform independent, Anaconda environment. You are encouraged to install the software on your




Based on the responses generated by our model, we can see that our algorithm tends to answer most of the questions with information apt with the contents of the ece447-OutlineW2024 document that we uploaded.The model seems to be very good at parsing and extracting the relevant information in-line with the nature of the queries. However, for some questions, we observed that the model also return some default internal prompts and responses along with giving the relevant information. The interface can also find it hard to decipher abbreviations/alternatives of the words mentioned in the document and used in queries.
Given the formatting of the PDF document, the model may find the relevant information but also return extra (irrelevant) information or give text as a response straight from the document itself without any altercation. Overall, it did seem to do a good job of unpacking the queries and extracting and returning the relevant information as expected with slight variances and/or extra information.

In [ ]:
# Write your answer and analysis here. (Boyong Wang Answered)

# For the question answer parts, for the very direct questions, for example, who is the professor? Who are TAs? It can provide the accurate answer. Another Example
# for direct questions is the evaluation scheme, it can also provide accurate answer. For the summarizing questions, for example, why student should learn this course?
# By the end of the course, student will be able to:, it can give the accurate summary. We can see that, Natural Language Processing System can Processing the language
# very well.
# However, for the senario analysis questions, sometimes it will not give more accurate answer. For example, if I get a 67% in this course, it give the answer that I can
# get a C+. However, using the marking scheme, because 67% is higher than 65%, and when I get 65%, I can get a B-,so I can also get a B level grade when I get 67%! Another
# example is, If I get 80%, but I am in the top 10% in the class, it provide me the answer that I can get an A+. However, using my analysis, it can be also A or A-, depend
# on the professor! For the senario analysis, humans will be doing better than Natural Language Processing Systems, because human will have more better analysis skills for
# Senario questions!

# Overall, the Natural Language Processing System, can answer direct questions, because it can understand the meaning of the language directly. However, for the senario
# Analysis questions, it can not provide accurate answers sometimes, and for senario questions, humans will have more better analysis skills than natural language processing
# systems.